### MHI_building: this notebook should be run first when starting from the beginning (or with new data).
This notebook reads in the "Dispositions" and "Sentencing" files and checks to see which rows should be marked as having a mental health indicator (MHI). For example, if a row (case_participant_id) in the Sentencing dataset has a value of "Inpatient Mental Health Services" in the column "Commitment Type", this row would be marked as MHI = True.

In [1]:
import pandas as pd
import numpy as np
import copy
import time

disposition = pd.read_pickle('dispositions.pckl.gz', compression = 'gzip')
sentencing = pd.read_pickle('sentencing.pckl.gz', compression = 'gzip')

In [2]:
# Just getting the columns that contain MHI indicators
sent_small = sentencing[['case_participant_id','charge_disposition', 'commitment_type', 
                         'charge_disposition_reason', 'sentence_type']]
disp_small = disposition[['case_participant_id', 'charge_disposition_reason', 'charge_disposition']]
disp_small = disp_small.rename(columns = {'charge_disposition_reason':'charge_disposition_reason_disp', 
                                          'charge_disposition': 'charge_disposition_disp'}) 
sent_disp = sent_small.append(disp_small,sort=True)

In [3]:
# Checking which rows should be marked as MHI true, this will take 5-10 minutes. 

df = copy.deepcopy(sent_disp)
#df.columns = map(str.lower, df.columns)

def reason_disp(series):
    return (series == 'Mental Health Graduate').any()

def disp(series):
    return series.isin(['FNG Reason Insanity',
                        'Finding Guilty But Mentally Ill',
                        'Plea of Guilty But Mentally Ill',
                        'Verdict Guilty But Mentally Ill',
                        'Sexually Dangerous Person',
                        ]).any()

def commitment_type(series):
    return series.isin(['Mental Health Probation',
                        'Inpatient Mental Health Services',
                        ]).any()

def charge_disp(series):
    return series.isin(['FNG Reason Insanity',
                        'Finding Guilty But Mentally Ill',
                        'Plea of Guilty But Mentally Ill',
                        'Verdict Guilty But Mentally Ill',
                        'Sexually Dangerous Person',
                        ]).any()

def charge_disp_reason(series):
    return (series == 'Mental Health Graduate').any()

def sentence_type(series):
    return (series == 'Inpatient Mental Health Services').any()

start_time = time.time()

grouped = df.groupby('case_participant_id').agg({'sentence_type': sentence_type, 
                                                 'charge_disposition_reason_disp': reason_disp,
                                                 'charge_disposition_disp': disp,
                                                 'commitment_type': commitment_type,
                                                 'charge_disposition': charge_disp,
                                                 'charge_disposition_reason': charge_disp_reason,
                                                 })

grouped['MHI'] = grouped.any(axis=1)

df = pd.merge(df, grouped['MHI'], left_on='case_participant_id', right_index=True)

print('total time: {}'.format(time.time() - start_time))

total time: 1002.0644447803497


In [4]:
# Exporting file with just case_participant_ids where MHI = True
MHI_true = df[df['MHI']]
MHI_true_small = MHI_true[['case_participant_id', 'MHI']]
MHI_dropdup = MHI_true_small.drop_duplicates()
MHI_dropdup.to_csv('MHI_true.csv') 

MHI_dropdup

,case_participant_id,MHI
186,221334022779,True
1356,252358604006,True
1601,239071986813,True
1918,248157954693,True
2027,249897124820,True
...,...,...
783820,1072550274183,True
784049,1072643142491,True
810783,1085421680925,True
812943,1087004311669,True
